# Imputation Kalman Model
> Imputation using Kalman Models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
#| default_exp kalman.imputation

In [ ]:
#| export
import pandas as pd
import numpy as np
from meteo_imp.kalman.model import KalmanModel
from meteo_imp.results import ImputationResult
from meteo_imp.utils import *
from fastcore.basics import store_attr, patch
from meteo_imp.data_preparation import StandardScaler

import torch
from torch import Tensor

## Imputation

In [ ]:
#| export
class KalmanImputation:
    """Imputation using a kalman model"""
    def __init__(self, data: pd.DataFrame,
                 model: KalmanModel = KalmanModel, # a subclass of KalmanModel to be used as model
                ):
        self.data = data
        self.train_idx = ~torch.tensor(self.data.isna().any(axis=1))
        
        train_data = torch.tensor(data.to_numpy())
        self.scaler = StandardScaler(train_data)
        train_data = self.scaler.transform(train_data)
        self.train_data = train_data
        
        self.T = torch.arange(self.data.shape[0])
        self.model = model(self.train_data)
    def fit(self, n_iter=10, lr=.1) -> 'KalmanImputation':
        """Fit model parameters"""
        times = self.T[self.train_idx]
        obs_test = self.train_data[self.train_idx]
        self.model.train(times, obs_test, n_iter, lr)
        return self

    def impute(self,
               pred_all = False, # If the dataset should be replaced by the model predictions
                                # or only the gaps imputed using the model
              ):
        """Impute data in tidy format using model"""
        # predict either no all dataset or only on part
        if pred_all:
            time_mask = self.T
            data_mask = torch.ones_like(self.train_idx, dtype=bool)
        else:
            time_mask = self.T[~self.train_idx]
            data_mask = ~self.train_idx

        pred = self.model.predict(time_mask)
        
        imp_mean = self.data.copy()
        mean = self.scaler.inverse_transform(pred.mean)
        imp_mean.iloc[data_mask, :] = mean.cpu().numpy()
        imp_mean = imp_mean.assign(time=self.T).melt('time', value_name = 'mean')
        
        # for observations std is 0
        imp_std = pd.DataFrame(np.zeros_like(self.data), columns=self.data.columns)
        # get the diagonal of the covariance matrices (the variance) and transform to std
        std = cov2std(pred.cov)
        std = self.scaler.inverse_transform_std(std)
        imp_std.iloc[data_mask, :] = std.cpu().numpy()
        imp_std = imp_std.assign(time=self.T).melt('time',value_name = 'std')
        
        return pd.merge(imp_mean, imp_std, on=['time', 'variable'])       

In [ ]:
from meteo_imp.data_preparation import MeteoDataTest

In [ ]:
reset_seed(1)
data = MeteoDataTest.generate_gpfa(2, 5).add_random_missing()

In [ ]:
data.data

,x0,x1
0,0.023263,NaN
1,0.219627,0.268028
2,-0.039892,0.063075
3,NaN,NaN
4,-0.645490,-0.144866


In [ ]:
k_imp = KalmanImputation(data.data)

In [ ]:
k_imp.train_data

tensor([[ 0.3586,     nan],
        [ 0.8847,  0.9976],
        [ 0.1895,  0.0048],
        [    nan,     nan],
        [-1.4328, -1.0024]])

In [ ]:
k_imp.train_data[k_imp.train_idx]

tensor([[ 0.8847,  0.9976],
        [ 0.1895,  0.0048],
        [-1.4328, -1.0024]])

In [ ]:
k_imp.fit(10, lr=0.1)

starting


  0%|          | 0/10 [00:00<?, ?it/s]

<__main__.KalmanImputation>

In [ ]:
k_imp.impute()

,time,variable,mean,std
0,0,x0,-0.031254,0.473841
1,1,x0,0.219627,0.000000
2,2,x0,-0.039892,0.000000
3,3,x0,-0.280452,0.514002
4,4,x0,-0.645490,0.000000
5,0,x1,0.109620,0.259775
6,1,x1,0.268028,0.000000
7,2,x1,0.063075,0.000000
8,3,x1,-0.000474,0.280925
9,4,x1,-0.144866,0.000000


In [ ]:
data.data.shape

In [ ]:
k_imp.impute(pred_all=True)

#### Result

In [ ]:
#| export
@patch
def to_result(self: KalmanImputation, data_compl, var_names=None, units=None, pred_all=False):
    return ImputationResult(self.impute(pred_all), data_compl, self.model.filter.get_info(var_names), units)

In [ ]:
X = np.hstack([np.arange(0,3.), np.arange(3., 0, -1)]).reshape(6, 1)

In [ ]:
res = k_imp.to_result(data.data_compl_tidy)

In [ ]:
res.display_results()

## Debug

In [ ]:
from meteo_imp.data import hai

In [ ]:
td = MeteoDataTest(hai)

In [ ]:
td.add_gap(10, 'TA', 10)

In [ ]:
i_hai = KalmanImputation(td.data)

In [ ]:
i_hai.fit()

## Export 

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()